In [1]:
import multiprocessing as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
import gc
import requests
from urllib.parse import urlparse

In [2]:
# input data: 주소, 건물 정보. 토지 정보는 자동으로 가져와야 함. 토지 정보를 자동으로 가져오려면 오리지널 토지특성정보 &
# 토지이용계획정보, 좌표는 api에서 다운로드해야함.

In [3]:
# 1. 오리지널 토지특성정보, 토지이용계획정보 & 공시지가
# 2. 좌표
# 3. 건물 정보

In [4]:
# 1. 오리지널 토지특성정보 & 토지이용계획정보

In [5]:
p_land_specs_df = pd.read_csv('./prepped_data/land_specs_ver_4.csv').drop(columns=['공시지가'])
print(p_land_specs_df.shape)
p_land_specs_df.head()

(8706295, 10)


,지번주소,년,지목명,토지면적,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면
0,서울특별시 강남구 개포동 100-0,2013,전,876.0,자연녹지지역,지정되지않음,전,완경사,부정형,맹지
1,서울특별시 강남구 개포동 100-0,2014,전,876.0,자연녹지지역,지정되지않음,전,완경사,부정형,맹지
2,서울특별시 강남구 개포동 100-0,2015,전,876.0,자연녹지지역,지정되지않음,전,완경사,부정형,맹지
3,서울특별시 강남구 개포동 100-0,2016,전,876.0,자연녹지지역,지정되지않음,전,완경사,부정형,맹지
4,서울특별시 강남구 개포동 100-0,2017,전,876.0,자연녹지지역,지정되지않음,전,완경사,부정형,맹지


In [6]:
p_land_plans_df = pd.read_csv('./토지이용계획정보/original_land_plans_mapped.csv')
print(p_land_plans_df.shape)
p_land_plans_df.head()

(5149472, 355)


,지번주소,년,(한강)오염행위_제한지역,(한강)폐기물매립시설_설치제한지역,4대문안,가로구역별_최고높이_제한지역,가스공급설비,가스공급시설,가축사육제한구역,개발제한구역,개발진흥지구,개발행위허가제한지역,건축선,건축용도지역기타,건축용도지역미분류,건축허가_착공제한지역,경관광장,경관녹지,경관지구,경관지구기타,고가도로,고도지구,고등학교,고속철도,고압선,공간시설미분류,공공공지,공공도서관,공공문화체육시설미분류,공공시설구역,공공시설용지,공공주택지구,공공지원민간임대주택_공급촉진지구,공공청사,공동구,공설화장시설,공용시설보호지구,공원,공원마을지구(공원집단시설지구),공원문화유산지구,공원자연보존지구,공원자연환경지구,공익용산지,공익임지,공장설립승인지역,공장설립제한지역,공항,공항소음피해예상지역,공항소음피해지역,공항시설보호지구,과밀억제권역,광로1류(폭_70M_이상),광로2류(폭_50M_70M),광로3류(폭_40M_50M),광역계획구역,광역복합환승센터,광장,교육환경보호구역,교차점광장,교통광장,교통운수시설미분류,국가산업단지,국가지정문화재구역,국가하천,국립공원,국민임대주택단지예정지구,국지도로,국토이용기타용도지구,국토이용용도지구기타,군사기지_및_군사시설_보호구역,군사시설_보호구역,궤도,근린공원,근린광장,근린상업지역,기업형임대주택_공급촉진지구,기타공공공지시설,기타공공청사시설,기타공원시설,기타교통시설,기타녹지시설,기타도로시설,기타도시공간시설,기타도시방재시설,기타문화시설,기타방송통신시설,기타보건위생시설,기타사회복지시설,기타수도시설,기타시장시설,기타열공급설비,기타용도지역지구기타,기타용도지역지구미분류,기타용지,기타유통및공급시설,기타유통업무설비,기타자동차정류장,기타전기공급설비,기타주차장시설,기타철도시설,기타폐기물처리시설,기타하수도시설,기타학교시설,기타환경기초시설,노외주차장,녹지,농수산물공판장및농수산물종합유통센터,대공방어협조구역,대공방어협조구역(위탁고도_54_236m),대공방어협조구역(위탁고도_77_257m),대로1류(폭_35M_40M),대로2류(폭_30M_35M),대로3류(폭_25M_30M),대학,도로,도로구역,도서관,도시개발구역,도시개발구역기타,도시고속도로,도시관리계획_입안중,도시기타용도지역지구기타,도시기타용도지역지구미분류,도시기타용도지역지구용도지역지구,도시자연공원,도시자연공원구역,도시지역,도시지역기타,도시철도,등록문화재구역,리모델링지구,묘지공원,문화공원,문화시설,문화재,문화재보존영향_검토대상구역,문화재보호구역,문화재보호구역기타,문화지구,미관지구,미관지구기타,박물관,방송통신시설,방수설비,방재지구,방화지구,배수시설,변전소(전원개발사업구역),변전시설,보건위생시설미분류,보전녹지지역,보전산지,보전임지,보조간선도로,보존지구,보행자전용도로,부설주차장,비오톱1등급,비행안전제1구역(전술),비행안전제2구역(전술),비행안전제2구역(지원),비행안전제3구역(전술),비행안전제3구역(지원),비행안전제4구역(전술),비행안전제4구역(지원),비행안전제5구역(전술),비행안전제5구역(지원),비행안전제6구역(전술),사고지,사방시설,사방지,사업지역기타,사업지역미분류,사회복지시설,산업개발진흥지구,산업기술단지,산업시설구역,상대보호구역,상대정화구역,상수원보호구역,상수원보호기타,생산녹지지역,생태_경관보전지역,생태_경관완충보전구역,생태_경관핵심보전구역,소공원,소로1류(폭_10M_12M),소로2류(폭_8M_10M),소로3류(폭_8M_미만),소하천,소하천구역,소하천예정지,수도공급시설,수변공원,수질오염방지시설,수평표면구역,시_도_생태_경관보전지역,시_도야생생물보호구역,시가지경관지구,시가지조성사업지역,시도지정문화재구역,시장,시장정비구역,시장정비구역기타,시험림구역,아파트지구,액화석유가스충전시설,야생생물보호구역,어린이공원,여객자동차터미널,역사도심,역사문화미관지구,역사문화특화경관지구,역사문화환경보존지역,연결녹지,연구개발특구,연구시설,열공급설비,온천공보호구역,온천원보호지구,완충녹지,용도구역기타,용도구역미분류,용도지구취락지구,운동장,원추표면구역,원형보존지,유류저장및송유설비,유수시설,유수지,유원지,유치원,유통단지,유통상업지역,유통업무설비,일단의공업용지조성사업지역,일단의주택단지조성사업지역,일반광장,일반도로,일반미관지구,일반산업단지,일반상업지역,일반주거지역,일반철도,임업용산지,자동차검사시설,자동차운전학원,자동차전용도로,자동차정류장,자연경관지구,자연공원용도지구기타,자연녹지지역,자연재해위험지구,장애물제한표면구역,재개발구역,재개발구역기타,재정비촉진지구,재정비촉진지구기타,재해위험지구기타,재활용시설,저류시설,전기공급설비,전용주거지역,전이표면구역,전통사찰보존구역,절대보호구역,절대정화구역,정비구역,정비구역기타,정비예정구역,제1종일반주거지역,제1종전용주거지역,제1종지구단위계획구역,제2종일반주거지역,제2종전용주거지역,제3종_구역,제3종일반주거지역,제방,제한보호구역,제한보호구역(방공기지___1km),제한보호구역(전술항공_5km),제한보호구역(후방지역_500m),조례로정한지역,조망가로미관지구,조망가로특화경관지구,종합운동장,종합의료시설,주간선도로,주거용지,주거환경개선지구,주요시설광장,주차장,주차장기타,주차환경개선지구,준공업지역,준보전산지,준주거지역,중로1류(폭_20M_25M),중로2류(폭_15M_20M),중로3류(폭_12M_15M),중심대광장,중심상업지역,중심지미관지구,중요시설물보존지구,중요시설물보호지구,중요시설물보호지구(공용),중요시설물보호지구(공항),중점경관관리구역,중학교,지구단위계획구역,지역특화발전특구,지원시설구역,지정문화재구역,지하광장,지하도로,진입표면구역,집단취락지구,집산도로,철도,철도보호지구,청사,청소년수련시설,체육공원,체육시설,초등학교,최고고도지구,취수시설,침수위험지구,택지개발예정지구,택지개발예정지구기타,토지거래계약에관한허가구역,토지구획정리사업지구기타,토지형질변경규제지역,통제보호구역,통제보호구역(민통선이남_300m),통제보호구역(방공기지_500m),특수도로(보행자전용도로),특정개발진흥지구,특정용도제한지구,특화경관지구,폐기물처리및재활용시설,폐기물처리시설,하수도,하수종말처리시설,하천,하천구역,하천미분류,학교,학교시설보호지구,학교이적지,학교환경위생_정화구역,현상변경허가_대상구역,홍수관리구역,화장장,환경정비구역
0,서울특별시 강남구 개포동 100-0,2014,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,서울특별시 강남구 개포동 100-0,2017,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [7]:
p_addr_coor_df = pd.read_csv('./prepped_data/addr_coor_data_ver_1.csv').rename(columns={'X좌표':'x좌표', 'Y좌표':'y좌표'})
print(p_addr_coor_df.shape)
p_addr_coor_df.head()

(626610, 4)


,지번주소,도로명주소,x좌표,y좌표
0,서울특별시 종로구 청운동 1-0,서울특별시 종로구 자하문로36길 16-14,953042.185946,1.954799e+06
1,서울특별시 종로구 청운동 3-0,서울특별시 종로구 자하문로36길 16-11,953033.999264,1.954773e+06
2,서울특별시 종로구 청운동 3-150,서울특별시 종로구 자하문로36길 16-5,953028.717755,1.954717e+06
3,서울특별시 종로구 청운동 4-1,서울특별시 종로구 자하문로36길 28-0,952962.680291,1.954697e+06
4,서울특별시 종로구 청운동 3-94,서울특별시 종로구 자하문로36길 28-0,952962.680291,1.954697e+06


In [8]:
#p_land_plans_df = p_land_plans_df.sort_values(['지번주소', '년']).drop_duplicates(subset=['지번주소'], keep='last')

In [9]:
# 2. 좌표

In [10]:
# 지번주소로 도로명주소와 좌표를 다운로드함. 출처 - 행정안전부 도로명주소 안내시스템 개발자센터

def fetch_coor_data(addr_list):
    roadname_url = 'https://www.juso.go.kr/addrlink/addrLinkApi.do?'
    roadname_apikey = 'devU01TX0FVVEgyMDIxMDEyNDExMDM0MDExMDczNjE='
    coor_base_url = 'https://www.juso.go.kr/addrlink/addrCoordApi.do?'
    coor_apikey = 'devU01TX0FVVEgyMDIxMDIyNjExMDQwMDExMDg1ODU='
    # 도로명주소 검색 api key 만료일 2021년 4월 24일
    # 좌표검색 api key 만료일 2021년 5월 27일
    #apikey 만료시 https://www.juso.go.kr/addrlink/devAddrLinkRequestWrite.do?returnFn=write&cntcMenu=URL 가서 새로 받을 것.
    
    coor_data_df = pd.DataFrame({'지번주소':[], '도로명주소':[], 'x좌표':[], 'y좌표':[]})
    
    for i, address in tqdm(enumerate(addr_list)):
        address_no_zero = address.replace('-0','')
        
        roadname_result = requests.get(roadname_url + 'confmKey=' + roadname_apikey + '&currentPage=1' + '&countPerPage=1'\
                                       + '&keyword=' + address_no_zero + '&resultType=json' + '&hstryYn=Y')
        roadname_info = roadname_result.json()['results']['juso']
        
        # 지번주소에 매칭하는 도로명주소가 없을 경우, 근처 지번주소의 도로명주소로 대체
        
        if roadname_info == []:
            coor_data_df.loc[i,'지번주소'] = address
            coor_data_df.loc[i,'도로명주소'] = np.nan
            coor_data_df.loc[i, 'x좌표'] = np.nan
            coor_data_df.loc[i, 'y좌표'] = np.nan
            continue
                                
        roadname_info = roadname_info[0]                                
        
        coor_result = requests.get(coor_base_url + 'confmKey=' + coor_apikey + '&admCd=' + roadname_info['admCd']\
                                   + '&rnMgtSn=' + roadname_info['rnMgtSn'] + '&udrtYn=' + roadname_info['udrtYn']\
                                   + '&buldMnnm=' + roadname_info['buldMnnm'] + '&buldSlno=' + roadname_info['buldSlno']\
                                   + '&resultType=json')
        
        coor_info = coor_result.json()['results']['juso'][0]
        
        coor_x = pd.to_numeric(coor_info['entX'])
        coor_y = pd.to_numeric(coor_info['entY'])
        
        roadname_addr = roadname_info['roadAddrPart1']
        
        coor_data_df.loc[i,'지번주소'] = address
        coor_data_df.loc[i,'도로명주소'] = roadname_addr
        coor_data_df.loc[i, 'x좌표'] = coor_x
        coor_data_df.loc[i, 'y좌표'] = coor_y
        
    
    
    return coor_data_df

In [11]:
basedir = './Training/training_data_ver_20/'

In [12]:
training_data_chunk = next(pd.read_csv(basedir + 'training_data_ver_20.csv', chunksize=1))

In [13]:
print(training_data_chunk.shape)
training_data_chunk.head()

(1, 279)


,지번주소,계약년월,거래금액(만원),건축년도,주택유형,토지면적,지목명,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면,건물유무,도로명주소,구,동,년,year_linear,month_sin,month_cos,건물연식,건물연식_카테고리화,xnorm,ynorm,x_2nd,y_2nd,xy,(한강)오염행위_제한지역,(한강)폐기물매립시설_설치제한지역,4대문안,가로구역별_최고높이_제한지역,가스공급설비,가축사육제한구역,개발제한구역,개발진흥지구,개발행위허가제한지역,건축선,건축용도지역기타,건축용도지역미분류,건축허가_착공제한지역,경관광장,경관녹지,경관지구,경관지구기타,고도지구,고속철도,공공공지,공공시설구역,공공주택지구,공공지원민간임대주택_공급촉진지구,공공청사,공용시설보호지구,공원,공원자연환경지구,공익용산지,공장설립승인지역,공장설립제한지역,공항소음피해예상지역,공항시설보호지구,과밀억제권역,광로1류(폭_70M_이상),광로2류(폭_50M_70M),광로3류(폭_40M_50M),광장,교육환경보호구역,교통광장,국가산업단지,국가지정문화재구역,국가하천,국립공원,국민임대주택단지예정지구,국지도로,국토이용용도지구기타,군사기지_및_군사시설_보호구역,근린공원,근린상업지역,기타공공공지시설,기타공원시설,기타교통시설,기타녹지시설,기타도로시설,기타도시공간시설,기타문화시설,기타자동차정류장,기타주차장시설,기타철도시설,기타폐기물처리시설,노외주차장,녹지,대공방어협조구역,대공방어협조구역(위탁고도_54_236m),대공방어협조구역(위탁고도_77_257m),대로1류(폭_35M_40M),대로2류(폭_30M_35M),대로3류(폭_25M_30M),대학,도로,도로구역,도서관,도시개발구역,도시개발구역기타,도시관리계획_입안중,도시기타용도지역지구기타,도시기타용도지역지구미분류,도시기타용도지역지구용도지역지구,도시자연공원,도시자연공원구역,도시지역,도시지역기타,도시철도,리모델링지구,묘지공원,문화공원,문화시설,문화재,문화재보존영향_검토대상구역,문화재보호구역,문화지구,미관지구,미관지구기타,박물관,방송통신시설,방수설비,방재지구,방화지구,보건위생시설미분류,보전산지,보전임지,보존지구,보행자전용도로,부설주차장,비오톱1등급,비행안전제2구역(전술),비행안전제2구역(지원),비행안전제3구역(전술),비행안전제3구역(지원),비행안전제4구역(전술),비행안전제4구역(지원),비행안전제5구역(전술),비행안전제5구역(지원),비행안전제6구역(전술),사고지,사방지,사업지역기타,사업지역미분류,사회복지시설,산업개발진흥지구,산업시설구역,상대보호구역,상대정화구역,상수원보호기타,생산녹지지역,소공원,소로1류(폭_10M_12M),소로2류(폭_8M_10M),소로3류(폭_8M_미만),소하천,소하천구역,수도공급시설,수평표면구역,시_도_생태_경관보전지역,시_도야생생물보호구역,시가지경관지구,시장,시장정비구역,아파트지구,어린이공원,여객자동차터미널,역사도심,역사문화미관지구,역사문화특화경관지구,역사문화환경보존지역,연결녹지,연구개발특구,열공급설비,온천공보호구역,완충녹지,용도구역기타,용도구역미분류,용도지구취락지구,원추표면구역,원형보존지,유수지,유원지,유통단지,유통상업지역,유통업무설비,일단의공업용지조성사업지역,일단의주택단지조성사업지역,일반도로,일반미관지구,일반산업단지,일반상업지역,일반주거지역,일반철도,임업용산지,자동차검사시설,자동차정류장,자연경관지구,자연녹지지역,자연재해위험지구,장애물제한표면구역,재개발구역,재개발구역기타,재정비촉진지구,재정비촉진지구기타,전기공급설비,전용주거지역,전이표면구역,전통사찰보존구역,절대보호구역,절대정화구역,정비구역,정비구역기타,정비예정구역,제1종일반주거지역,제1종전용주거지역,제1종지구단위계획구역,제2종일반주거지역,제2종전용주거지역,제3종_구역,제3종일반주거지역,제한보호구역,제한보호구역(방공기지___1km),제한보호구역(전술항공_5km),제한보호구역(후방지역_500m),조망가로미관지구,조망가로특화경관지구,종합의료시설,주간선도로,주거환경개선지구,주차장,주차장기타,주차환경개선지구,준공업지역,준보전산지,준주거지역,중로1류(폭_20M_25M),중로2류(폭_15M_20M),중로3류(폭_12M_15M),중심지미관지구,중요시설물보호지구,중요시설물보호지구(공용),중요시설물보호지구(공항),중점경관관리구역,중학교,지구단위계획구역,지역특화발전특구,지원시설구역,지하도로,진입표면구역,집단취락지구,철도,철도보호지구,청소년수련시설,체육공원,체육시설,초등학교,최고고도지구,침수위험지구,택지개발예정지구,택지개발예정지구기타,토지거래계약에관한허가구역,통제보호구역(민통선이남_300m),특수도로(보행자전용도로),특정개발진흥지구,특정용도제한지구,특화경관지구,폐기물처리시설,하수도,하천,하천구역,학교,학교시설보호지구,학교이적지,학교환경위생_정화구역,현상변경허가_대상구역,면적당단가
0,서울특별시 강남구 대치동 891-23,201404,22000,2004.0,0,43.39,0,0,0,0,0,0,0,1,서울특별시 강남구 선릉로90길 56-0,0,0,2014,1.254795,1.0,6.123234e-17,10.0,0,7244.178842,-6450.462242,2.623906e+07,2.080423e+07,-4.672830e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,507.029269


In [14]:
cols_to_drop = ['지번주소', '계약년월', '거래금액(만원)', '건축년도', '토지면적', '도로명주소', '년', '건물연식', '면적당단가']

In [15]:
training_features = training_data_chunk.columns.drop(cols_to_drop)
training_features[:10]

Index(['주택유형', '지목명', '용도지역명1', '용도지역명2', '토지이동상황', '지형높이', '지형형상', '도로접면',
       '건물유무', '구'],
      dtype='object')

In [16]:
import lightgbm as lgb

In [17]:
mse_model_names_list = [f for f in os.listdir(basedir) if 'lightgbm_huber' in f]
mse_model_names_list

['lightgbm_huber_0.txt',
 'lightgbm_huber_1.txt',
 'lightgbm_huber_2.txt',
 'lightgbm_huber_3.txt',
 'lightgbm_huber_4.txt']

In [18]:
mse_models_list = []
for f in tqdm(mse_model_names_list):
    model = lgb.Booster(model_file=basedir + f)
    mse_models_list.append(model)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:40<00:00,  8.16s/it]


In [19]:
def month_circular_sine_func(x):
    return math.sin(2*math.pi*((x-1)/12))

def month_circular_cosine_func(x):
    return math.cos(2*math.pi*((x-1)/12))

In [20]:
def year_linear_feature_engi(x_month):
    
    days_in_months = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    
    days_sum = 0
    for i in range(x_month - 1):
        days_sum += days_in_months[i]
    
    return days_sum

In [21]:
def datetime_feature_engi(df):
    df = df.copy()
    
    #current_date = datetime.datetime.now()
    
    year = df['기준시점'].apply(lambda x: int(str(x).split('.')[0]))
    month = df['기준시점'].apply(lambda x: int(str(x).split('.')[1]))
    day = df['기준시점'].apply(lambda x: int(str(x).split('.')[2]))
    
    df['year_linear'] = (year - 2013) + (month.apply(year_linear_feature_engi) + (day/365))    
    
    df['month_sin'] = month.apply(month_circular_sine_func)
    df['month_cos'] = month.apply(month_circular_cosine_func)
    
    return df

In [22]:
def coor_feature_engi(coor_df, coor_minmax_df):
    coor_df = coor_df.copy()
    coor_minmax_df = coor_minmax_df.copy()
    
    coor_df['xnorm'] = (coor_df['x좌표'] - coor_minmax_df['xmin'][0] - ((coor_minmax_df['xmax'][0] - coor_minmax_df['xmin'][0])/2))
    coor_df['ynorm'] = (coor_df['y좌표'] - coor_minmax_df['ymin'][0] - ((coor_minmax_df['ymax'][0] - coor_minmax_df['ymin'][0])/2))
    coor_df['x_2nd'] = coor_df['xnorm']**2 / 2
    coor_df['y_2nd'] = coor_df['ynorm']**2 / 2
    coor_df['xy'] = coor_df['xnorm'] * coor_df['ynorm']
    
    coor_df = coor_df.drop(columns=['x좌표', 'y좌표'])
    
    return coor_df

In [42]:
def prediction_pipeline(models_list, manual_input_df, land_specs_df, land_plans_df, selected_features_list,
                        addr_coor_df, verbose=1):
    manual_input_df['년'] = manual_input_df['기준시점'].apply(lambda x: int(str(x).split('.')[0]))
    
    prediction_data_df = manual_input_df.merge(land_specs_df, on=['지번주소', '년'], how='left')\
    .merge(land_plans_df, on=['지번주소', '년'], how='left')
        
    addr_list = manual_input_df['지번주소'].unique().tolist()
    
    selected_addr_coor_df = addr_coor_df[addr_coor_df['지번주소'].isin(addr_list)]
    
    #return selected_addr_coor_df, addr_list
    
    if selected_addr_coor_df.shape[0] == len(addr_list):
        coor_df = selected_addr_coor_df[['지번주소', 'x좌표', 'y좌표']]
    else:    
        selected_addr_list = selected_addr_coor_df['지번주소'].unique().tolist()
        for addr_to_drop in selected_addr_list:
            addr_list.remove(addr_to_drop)        
        coor_df = fetch_coor_data(addr_list)
        coor_df = coor_df[['지번주소', 'x좌표', 'y좌표']]
    
    coor_minmax_df = pd.read_csv('./Training/training_data_ver_20/coor_minmax.csv')
    
    coor_df = coor_feature_engi(coor_df, coor_minmax_df)
    
    prediction_data_df = prediction_data_df.merge(coor_df, on=['지번주소'], how='left')
    
    prediction_data_df = datetime_feature_engi(prediction_data_df)
    
    prediction_data_df = prediction_data_df[selected_features_list]
        
    cat_cols_df = pd.read_csv('./Training/training_data_ver_20/cat_cols.csv')
    
    cat_cols = []
    for col in cat_cols_df['colname']:
        if col in prediction_data_df.columns:
            if col not in land_plans_df.columns:
                cat_cols.append(col)
    
    if verbose == 1:
        print('mapping categorical data...')
    
        for col in tqdm(cat_cols):
            col_map_df = pd.read_csv('./Training/training_data_ver_20/feature_maps/{}_map.csv'.format(col))
            col_map = {}
            for i in range(col_map_df.shape[0]):
                col_map[col_map_df['cat'][i]] = col_map_df['map'][i]
            prediction_data_df[col] = prediction_data_df[col].map(col_map)
    
    elif verbose == 0:
        for col in cat_cols:
            col_map_df = pd.read_csv('./Training/training_data_ver_20/feature_maps/{}_map.csv'.format(col))
            col_map = {}
            for i in range(col_map_df.shape[0]):
                col_map[col_map_df['cat'][i]] = col_map_df['map'][i]
            prediction_data_df[col] = prediction_data_df[col].map(col_map)
        
    
    return prediction_data_df

In [24]:
reference_df = pd.read_excel('./Production_Prep/이승준_총괄표.xlsx', header=1)
print(reference_df.shape)
reference_df.head()

(137, 39)


,지역,소재지,지번,용도지역,도로너비,토지면적(㎡),토지매매금액,토지매매단가\n(원/㎡),공시지가와격차\n(개공/매매),건물용도,공급면적(㎡),전유면적(㎡),전용율,용적율\n(%),건폐율\n(%),구분건물호,건물동수,건물층수,의뢰인,기준시점,전체평가금액,투입비용합계,순이익,순이익율,전유면적,공급면적,전유면적기준,공급면적기준,토지계약일자,허가일자,착공일자,사용승인일,Unnamed: 32,토지원가비율,건축원가비율,부대비용,금융비용,합계,Unnamed: 38
0,강동구,천호동,17-4외,2종일주,4~6미터,934.0,5.549000e+09,5941113.0,0.541,도시형생활주택,1720.57,1418.72,0.825,184.22,55.98,27개호,3개동,지상5층,SH공사,2019.12.18,1.005050e+10,9.128694e+09,921806160.0,0.092,7084203.0,5.841088e+06,6.434458e+06,5515021.0,2019.04.04,2019.11.26,2019.12.26,2020.07.09,5500000.0,0.6473,0.3233,0.0091,0.0203,1.0,NaN
1,강동구,천호동,18-65외,2종일반주거,6미터,261.0,1.880000e+09,7203065.0,0.484,도시형생활주택,517.56,414.78,0.689,199.83,53.57,15개호,1개동,지하1\n지상6,LH공사,2019.06.07,3.633000e+09,3.109518e+09,523482390.0,0.144,8758860.0,6.034783e+06,7.496788e+06,5932890.0,2018.06.13,2018.09.18,2018.10.19,2019.04.11,5400000.0,0.6423,0.3262,0.0091,0.0224,1.0,NaN
2,강동구,천호동,18-163외,2종일반주거,4미터,228.0,1.510000e+09,6622807.0,0.490,도시형생활주택,454.94,378.30,0.832,199.54,58.35,14세대,1개동,14세대,SH공사,2020.11.23,3.091500e+09,2.448262e+09,643238380.0,0.208,8172086.0,6.795402e+06,6.471746e+06,5588504.0,2019.09.18,2020.04.24,2020.05.21,2020.11.11,5400000.0,0.6552,0.3131,0.0088,0.0229,1.0,NaN
3,강동구,천호동,36-8외,2종일주,4미터미만,267.4,1.860000e+09,6955871.0,0.530,다세대주택,534.75,467.37,0.874,199.54,58.35,10세대,1개동,10세대,SH공사,2020.10.23,3.232500e+09,2.992845e+09,239655010.0,0.074,6916362.0,6.044881e+06,6.403588e+06,5827722.0,2019.10.30,2020.02.10,2020.03.30,2020.10.08,5500000.0,0.6612,0.3065,0.0086,0.0237,1.0,NaN
4,강동구,암사동,433-69외,3종일주,4미터,258.0,1.643000e+09,6368217.0,0.479,다세대주택,493.67,408.01,0.850,209.57,58.46,9세대,1개동,지상5층,SH공사,2020.10.23,2.901000e+09,2.657727e+09,243272910.0,0.084,7110120.0,6.040855e+06,6.513877e+06,5319617.0,2019.11.11,2020.02.14,2020.05.04,2020.09.28,5500000.0,0.6573,0.3100,0.0087,0.0241,1.0,NaN


In [25]:
reference_df.isna().sum()

지역                    5
소재지                   6
지번                    6
용도지역                  6
도로너비                  6
토지면적(㎡)               6
토지매매금액                6
토지매매단가\n(원/㎡)         6
공시지가와격차\n(개공/매매)      5
건물용도                  6
공급면적(㎡)               6
전유면적(㎡)               6
전용율                   5
용적율\n(%)              6
건폐율\n(%)              6
구분건물호                 6
건물동수                  6
건물층수                  6
의뢰인                   6
기준시점                  6
전체평가금액                6
투입비용합계                6
순이익                   6
순이익율                  5
전유면적                  5
공급면적                  5
전유면적기준                5
공급면적기준                5
토지계약일자                6
허가일자                  6
착공일자                  6
사용승인일                 6
Unnamed: 32           6
토지원가비율                5
건축원가비율                5
부대비용                  5
금융비용                  5
합계                    5
Unnamed: 38         132
dtype: int64

In [26]:
reference_df['토지면적단가'] = reference_df['전체평가금액'] / reference_df['토지면적(㎡)']

In [27]:
reference_df['토지면적단가'] = reference_df['토지면적단가'] / 10000

In [28]:
def jibun_modifier(x):
    # x is a string
    
    if pd.isna(x) == True:
        return x
    else:
        x = str(x)
        x = x.replace('외', '')
        splitted = x.split('-')
        if len(splitted) == 1:
            return x + '-0'
        else:
            return x

In [29]:
reference_df['지번주소'] = '서울특별시 ' + reference_df['지역'] + ' ' + reference_df['소재지'] + ' ' + reference_df['지번'].apply(jibun_modifier)

In [30]:
reference_df['예측토지면적단가'] = np.nan

In [31]:
def date_modifier(x):
    # x is a string or datetime
    if pd.isna(x) == True:
        return x
    else:
        x = str(x)
        if '-' in x:
            x = x.replace('-', '.')
            
        splitted = x.split('.')
        
        if len(splitted) == 1:
            return np.nan
        else:
            if ':' in x:
                return x.split(' ')[0]
            else:
                return x

In [32]:
reference_df['기준시점'] = reference_df['기준시점'].apply(date_modifier)

In [33]:
reference_df['기준시점'].unique()

array(['2019.12.18', '2019.06.07', '2020.11.23', '2020.10.23',
       '2019.04.01', '2019.06.14', '2020.05.25', '2020.11.11',
       '2020.07.20', '2020.09.23', '2019.11.29', '2019.12.27',
       '2020.10.13', '2020.03.10', '2020.01.10', '2020.10.06',
       '2020.10.30', '2020.07.23', '2020.06.25', '2020.09.04',
       '2020.12.05', '2020.04.27', '2020.05.27', '2020.12.27',
       '2020.08.27', '2019.11.12', '2019.11.22', '2019.04.22',
       '2020.06.18', '2019.01.28', '2019.09.10', '2019.01.03',
       '2019.01.18', '2019.12.13', '2019.05.27', '2019.05.14',
       '2019.03.04', '2020.08.13', '2020.07.10', '2020.06.08',
       '2020.07.17', '2020.01.28', '2020.09.21', '2020.12.10', nan,
       '2019.06.24', '2018.03.23', '2019.12.06', '2019.06.20',
       '2019.10.23', '2019.12.20', '2020.11.30', '2020.12.02'],
      dtype=object)

In [34]:
reference_df.isna().sum()

지역                    5
소재지                   6
지번                    6
용도지역                  6
도로너비                  6
토지면적(㎡)               6
토지매매금액                6
토지매매단가\n(원/㎡)         6
공시지가와격차\n(개공/매매)      5
건물용도                  6
공급면적(㎡)               6
전유면적(㎡)               6
전용율                   5
용적율\n(%)              6
건폐율\n(%)              6
구분건물호                 6
건물동수                  6
건물층수                  6
의뢰인                   6
기준시점                 51
전체평가금액                6
투입비용합계                6
순이익                   6
순이익율                  5
전유면적                  5
공급면적                  5
전유면적기준                5
공급면적기준                5
토지계약일자                6
허가일자                  6
착공일자                  6
사용승인일                 6
Unnamed: 32           6
토지원가비율                5
건축원가비율                5
부대비용                  5
금융비용                  5
합계                    5
Unnamed: 38         132
토지면적단가                6
지번주소                  6
예측토지면적단가        

In [35]:
reference_df = reference_df.dropna(subset=['지번주소', '기준시점']).reset_index(drop=True)
reference_df.shape

(86, 42)

In [36]:
reference_df.head()

,지역,소재지,지번,용도지역,도로너비,토지면적(㎡),토지매매금액,토지매매단가\n(원/㎡),공시지가와격차\n(개공/매매),건물용도,공급면적(㎡),전유면적(㎡),전용율,용적율\n(%),건폐율\n(%),구분건물호,건물동수,건물층수,의뢰인,기준시점,전체평가금액,투입비용합계,순이익,순이익율,전유면적,공급면적,전유면적기준,공급면적기준,토지계약일자,허가일자,착공일자,사용승인일,Unnamed: 32,토지원가비율,건축원가비율,부대비용,금융비용,합계,Unnamed: 38,토지면적단가,지번주소,예측토지면적단가
0,강동구,천호동,17-4외,2종일주,4~6미터,934.0,5.549000e+09,5941113.0,0.541,도시형생활주택,1720.57,1418.72,0.825,184.22,55.98,27개호,3개동,지상5층,SH공사,2019.12.18,1.005050e+10,9.128694e+09,921806160.0,0.092,7084203.0,5.841088e+06,6.434458e+06,5515021.0,2019.04.04,2019.11.26,2019.12.26,2020.07.09,5500000.0,0.6473,0.3233,0.0091,0.0203,1.0,NaN,1076.070664,서울특별시 강동구 천호동 17-4,NaN
1,강동구,천호동,18-65외,2종일반주거,6미터,261.0,1.880000e+09,7203065.0,0.484,도시형생활주택,517.56,414.78,0.689,199.83,53.57,15개호,1개동,지하1\n지상6,LH공사,2019.06.07,3.633000e+09,3.109518e+09,523482390.0,0.144,8758860.0,6.034783e+06,7.496788e+06,5932890.0,2018.06.13,2018.09.18,2018.10.19,2019.04.11,5400000.0,0.6423,0.3262,0.0091,0.0224,1.0,NaN,1391.954023,서울특별시 강동구 천호동 18-65,NaN
2,강동구,천호동,18-163외,2종일반주거,4미터,228.0,1.510000e+09,6622807.0,0.490,도시형생활주택,454.94,378.30,0.832,199.54,58.35,14세대,1개동,14세대,SH공사,2020.11.23,3.091500e+09,2.448262e+09,643238380.0,0.208,8172086.0,6.795402e+06,6.471746e+06,5588504.0,2019.09.18,2020.04.24,2020.05.21,2020.11.11,5400000.0,0.6552,0.3131,0.0088,0.0229,1.0,NaN,1355.921053,서울특별시 강동구 천호동 18-163,NaN
3,강동구,천호동,36-8외,2종일주,4미터미만,267.4,1.860000e+09,6955871.0,0.530,다세대주택,534.75,467.37,0.874,199.54,58.35,10세대,1개동,10세대,SH공사,2020.10.23,3.232500e+09,2.992845e+09,239655010.0,0.074,6916362.0,6.044881e+06,6.403588e+06,5827722.0,2019.10.30,2020.02.10,2020.03.30,2020.10.08,5500000.0,0.6612,0.3065,0.0086,0.0237,1.0,NaN,1208.863126,서울특별시 강동구 천호동 36-8,NaN
4,강동구,암사동,433-69외,3종일주,4미터,258.0,1.643000e+09,6368217.0,0.479,다세대주택,493.67,408.01,0.850,209.57,58.46,9세대,1개동,지상5층,SH공사,2020.10.23,2.901000e+09,2.657727e+09,243272910.0,0.084,7110120.0,6.040855e+06,6.513877e+06,5319617.0,2019.11.11,2020.02.14,2020.05.04,2020.09.28,5500000.0,0.6573,0.3100,0.0087,0.0241,1.0,NaN,1124.418605,서울특별시 강동구 암사동 433-69,NaN


In [37]:
reference_df['건물용도'].unique()

array(['도시형생활주택', '다세대주택', '다세대-도시형주택', '도생-원룸형', '단지형다세대', '도생(원룸형)',
       '도생(단지형다세대)', '도시행생활주택', '도생-원룸', '도생-다세대', '도생', '도생-단지형다세대',
       '다세대', '도새-원룸', '오피스텔 및 근생', '도생-단지형'], dtype=object)

In [38]:
def bd_purpose(x):
    # x is a stirng
    if pd.isna(x) == True:
        return x
    else:
        if '다세' in x:
            return '연립다세대'
        elif '오피' in x:
            return '오피스텔'
        elif ('도시' in x)|('도생' in x):
            return '연립다세대'
        else:
            return '연립다세대'

In [39]:
reference_df['주택유형'] = reference_df['건물용도'].apply(bd_purpose)

In [40]:
reference_df['주택유형'].value_counts()

연립다세대    85
오피스텔      1
Name: 주택유형, dtype: int64

In [46]:
for i in tqdm(range(reference_df.shape[0])):
    addr = reference_df.loc[i, '지번주소']
    r_date = reference_df.loc[i, '기준시점']
    gu = reference_df.loc[i, '지역']
    dong = reference_df.loc[i, '소재지']
    bd_type = reference_df.loc[i, '주택유형']
    
    if (pd.isna(addr) == True):
        continue
    
    manual_input_df = pd.DataFrame({'지번주소':[addr], '건물연식_카테고리화':['0년'], '기준시점':[r_date], '구':[gu], '동':[dong],
                                    '주택유형':[bd_type], '건물유무':[1]})
    
    prac_df = prediction_pipeline(mse_models_list, manual_input_df, p_land_specs_df, p_land_plans_df, training_features,
                                  p_addr_coor_df, verbose=0)
    
    if prac_df.isna().sum().sum() > 0:
        print('error occured')
        continue
    
    print(prac_df.shape)
    print('making predictions...')
    preds_list = []
    for model in mse_models_list:
        preds = model.predict(prac_df)
        preds_list.append(preds)
    
    print('prediction complete')
    
    reference_df.loc[i, '예측토지면적단가'] = np.mean(preds_list)

  0%|                                                                                           | 0/86 [00:00<?, ?it/s]

(1, 270)
making predictions...


  1%|▉                                                                                | 1/86 [01:12<1:42:02, 72.03s/it]

prediction complete
(1, 270)
making predictions...


  2%|█▉                                                                               | 2/86 [02:25<1:42:09, 72.97s/it]

...

prediction complete
(1, 270)
making predictions...


 99%|███████████████████████████████████████████████████████████████████████████████ | 85/86 [1:28:59<01:23, 83.78s/it]

prediction complete
(1, 270)
making predictions...


100%|████████████████████████████████████████████████████████████████████████████████| 86/86 [1:30:46<00:00, 63.33s/it]

prediction complete


In [47]:
reference_df[['토지면적단가', '예측토지면적단가']].isna().sum()

토지면적단가       0
예측토지면적단가    45
dtype: int64

In [48]:
nonan_df = reference_df.dropna(subset=['토지면적단가', '예측토지면적단가'])

In [49]:
from sklearn.metrics import mean_squared_error

In [50]:
np.sqrt(mean_squared_error(nonan_df['토지면적단가'], nonan_df['예측토지면적단가']))

295.8753628465621

In [51]:
#reference_df.to_excel('./Production_Prep/이승준_총괄표_토지단가_예측포함.xlsx')

In [52]:
inv_df = reference_df[['지번주소', '건물용도', '주택유형', '의뢰인', '기준시점', '토지면적단가', '예측토지면적단가']]
inv_df.shape

(86, 7)

In [53]:
inv_df

,지번주소,건물용도,주택유형,의뢰인,기준시점,토지면적단가,예측토지면적단가
0,서울특별시 강동구 천호동 17-4,도시형생활주택,연립다세대,SH공사,2019.12.18,1076.070664,1256.208634
1,서울특별시 강동구 천호동 18-65,도시형생활주택,연립다세대,LH공사,2019.06.07,1391.954023,1513.278512
2,서울특별시 강동구 천호동 18-163,도시형생활주택,연립다세대,SH공사,2020.11.23,1355.921053,NaN
3,서울특별시 강동구 천호동 36-8,다세대주택,연립다세대,SH공사,2020.10.23,1208.863126,1226.474935
4,서울특별시 강동구 암사동 433-69,다세대주택,연립다세대,SH공사,2020.10.23,1124.418605,NaN
5,서울특별시 강동구 암사동 454-26,다세대주택,연립다세대,SH공사,2019.04.01,1116.701147,1430.460725
6,서울특별시 강동구 암사동 455-51,다세대주택,연립다세대,LH공사,2019.04.01,1235.948645,1576.674885
7,서울특별시 강동구 암사동 457-30,다세대주택,연립다세대,SH공사,2019.06.14,1072.149644,1530.821486
8,서울특별시 강동구 고덕동 292-5,다세대주택,연립다세대,LH공사,2019.06.14,1460.648148,1688.802268
9,서울특별시 강동구 고덕동 292-6,다세대주택,연립다세대,SH공사,2019.06.14,1463.029067,1680.062146


In [ ]:
inv_df['오차'] = inv_df['토지면적단가'] - inv_df['예측토지면적단가']

In [ ]:
inv_df['오차율'] = inv_df['오차'] / inv_df['토지면적단가'] * 100

In [ ]:
inv_df.rename(columns={'오차율':'오차율(%)'}, inplace=True)

In [ ]:
dasede_df = inv_df[inv_df['건물용도'].str.contains('다세')]
print(dasede_df.shape)
dasede_df.head()

In [ ]:
np.sqrt(mean_squared_error(dasede_df['토지면적단가'], dasede_df['예측토지면적단가']))

In [ ]:
dasede_df['오차율절대값'] = dasede_df['오차율(%)'].apply(np.abs)

In [ ]:
dasede_df = dasede_df.sort_values(['오차율절대값'], ascending=False)

In [ ]:
dasede_df

In [ ]:
dasede_df['오차율절대값'].mean()